### Usando Visualizações como Consultas Dinâmicas

Embora os widgets de interface padrão mostrem os valores _possíveis_ para os parâmetros de consulta, eles não visualizam a _distribuição_ desses valores. Também podemos desejar usar interações mais ricas, como seleções de vários valores ou intervalos, em vez de widgets de entrada que selecionam apenas um único valor de cada vez.

Para resolver esses problemas, podemos criar gráficos adicionais para visualizar dados e oferecer suporte a consultas dinâmicas. Vamos adicionar um histograma da contagem de filmes por ano e utilizar uma seleção de intervalo para realçar dinamicamente os filmes em períodos de tempo selecionados.

*Interaja com o histograma de anos para explorar filmes de diferentes períodos de tempo. Vê alguma evidência de [viés de amostragem] (https://en.wikipedia.org/wiki/Sampling_bias) ao longo dos anos? (Qual é a relação entre o ano e as classificações dos críticos?)*.

_Os anos vão de 1930 a 2040! Os futuros filmes estão em pré-produção ou  existem erros "fora de um século"? Além disso, dependendo do fuso horário em que está, irá ver um pequeno aumento em 1969 ou 1970. Porque isso acontece? (Veja o final do notebook para uma explicação!)_


In [ ]:
brush = alt.selection_interval(
    encodings=['x'] # limitar a seleção aos valores do eixo x (ano)
)

# histograma de consulta dinâmico
years = alt.Chart(movies).mark_bar().add_selection(
    brush
).encode(
    alt.X('year(Release_Date):T', title='Films by Release Year'),
    alt.Y('count():Q', title=None)
).properties(
    width=650,
    height=50
)

# scatter plot, modificar a opacidade com base na seleção
ratings = alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q',
    tooltip='Title:N',
    opacity=alt.condition(brush, alt.value(0.75), alt.value(0.05))
).properties(
    width=650,
    height=400
)

alt.vconcat(years, ratings).properties(spacing=5)

alt.VConcatChart(...)

O exemplo acima fornece consultas dinâmicas utilizando uma _seleção vinculada_ entre gráficos:

- Criamos uma seleção `interval` (`brush`), e definimos `encodings=['x']` para limitar a seleção apenas ao eixo x, resultando num intervalo de seleção unidimensional.
- Registamos o `brush` com o nosso histograma de filmes por ano através de `.add_selection(brush)`.
- Usamos `brush` numa codificação condicional para ajustar a opacidade (`opacity`) do scatter plot.

Esta técnica de interação de selecionar elementos num gráfico e ver os destaques ligados num ou mais gráficos é conhecida como [_brushing &amp; linking_](https://en.wikipedia.org/wiki/Brushing_and_linking).

## Panning &amp; Zooming

O _scatter plot_ da classificação dos filmes é um pouco confuso em alguns sítios, tornando difícil examinar pontos em regiões mais densas. Utilizando as técnicas de interação de _panning_ e _zooming_, podemos inspecionar regiões densas mais de perto.

Comecemos pensando em alguma forma de expressar o _panning_ e o _zooming_ utilizando as seleções do Altair. O que define a "janela de visualização" de um gráfico? _Domínios de escala dos eixos!_

Podemos alterar os domínios de escala para modificar o intervalo visualizado de valores de dados. Para fazer isso interativamente, podemos vincular uma seleção `interval` aos domínios de escala com o código `bind='scales'`. O resultado é que, em vez de um _brush_ de intervalo que podemos arrastar e ampliar, podemos arrastar e ampliar toda a área de plotagem!

No gráfico abaixo, clique e arraste para deslocar (traduzir) a visualização, ou role para ampliar (escalar) a visualização. O que é que pode descobrir sobre a precisão dos valores de classificação fornecidos?_


In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales')
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

Ao fazer _zooming_, podemos ver que os valores de classificação têm uma precisão limitada! As classificações do Rotten Tomatoes são números inteiros, enquanto as classificações do IMDB são truncadas em décimos. Como resultado, existe uma sobreposição de gráficos mesmo quando fazemos zoom, com vários filmes compartilhando os mesmos valores de classificação.

Lendo o código acima, você pode notar o código `alt.Axis(minExtent=30)` no canal de codificação `y`. O parâmetro `minExtent` garante que uma quantidade mínima de espaço seja reservada para os ticks e rótulos do eixo. Por que fazer isso? Quando fazemos pan e zoom, os rótulos dos eixos podem mudar e fazer com que a posição do título do eixo se desloque. Ao definir uma extensão mínima, podemos reduzir os movimentos de distração no gráfico. Tente alterar o valor de `minExtent`, por exemplo, definindo-o como zero, e depois diminua o zoom para ver o que acontece quando rótulos de eixo mais longos entram na visualização.

O Altair também inclui uma forma abreviada de adicionar _panning_ e _zooming_ a um gráfico. Ao invés de criar diretamente uma seleção, você pode chamar `.interactive()` para que o Altair gere automaticamente uma seleção de intervalos vinculada às escalas do gráfico:


In [ ]:
alt.Chart(movies).mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
).interactive()

alt.Chart(...)

Por padrāo, as ligações de escala para seleções incluem os canais de codificação `x` e `y`. E se quisermos limitar o _panning_ e o _zoomimg_ ao longo de uma única dimensão? Podemos invocar `encodings=['x']` para restringir a seleção apenas ao canal `x`:


In [ ]:
alt.Chart(movies).mark_circle().add_selection(
    alt.selection_interval(bind='scales', encodings=['x'])
).encode(
    x='Rotten_Tomatoes_Rating:Q',
    y=alt.Y('IMDB_Rating:Q', axis=alt.Axis(minExtent=30)), # utilizar a extensão mínima para estabilizar a colocação do título do eixo
    tooltip=['Title:N', 'Release_Date:N', 'IMDB_Rating:Q', 'Rotten_Tomatoes_Rating:Q']
).properties(
    width=600,
    height=400
)

alt.Chart(...)

_Quando se faz zooming apenas ao longo de um único eixo, a forma dos dados visualizados pode mudar, afetando potencialmente a nossa perceção das relações nos dados. [Escolhendo uma proporção de aspecto adequada](http://vis.stanford.edu/papers/arclength-banking) é uma preocupação importante do design de visualização!_


## Navegação: Visão Geral + Detalhe

Ao usar _panning_ e _zooming_, ajustamos diretamente a "janela de visualização" de um gráfico. A estratégia de navegação relacionada de _visão geral + detalhe_ utiliza, em vez disso, uma visão geral de exibição para mostrar _todos_ os dados, ao mesmo tempo que suporta seleções pan e zoom em uma exibição de foco separado.

Abaixo temos dois gráficos de área que mostram uma década de flutuações de preços para o índice de ações S&amp;P 500. Inicialmente, os dois gráficos mostram o mesmo intervalo de dados. Clique e arraste no gráfico de visão geral inferior para atualizar a exibição de foco e examinar intervalos de tempo específicos.


In [ ]:
brush = alt.selection_interval(encodings=['x']);

base = alt.Chart().mark_area().encode(
    alt.X('date:T', title=None),
    alt.Y('price:Q')
).properties(
    width=700
)

alt.vconcat(
    base.encode(alt.X('date:T', title=None, scale=alt.Scale(domain=brush))),
    base.add_selection(brush).properties(height=60),
    data=sp500
)

alt.VConcatChart(...)

Ao contrário do nosso caso anterior, panning &amp; zooming, aqui não queremos vincular uma seleção diretamente às escalas de um único gráfico interativo. Em vez disso, queremos vincular a seleção a um domínio de escala em _outro_ gráfico. Para isso, atualizamos o canal de codificação `x` do nosso gráfico de foco, definindo a propriedade domínio (`domain`) da escala para fazer referência à nossa seleção `brush`. Se nenhum intervalo for definido (a seleção está vazia), o Altair ignora o _brush_ e usa os dados subjacentes para determinar o domínio. Quando um intervalo de brush é criado, o Altair o utiliza como o domínio (`domain`) da escala para o gráfico de foco.

## Detalhes a Pedido

Uma vez identificados os pontos de interesse numa visualização, queremos frequentemente saber mais sobre esses pontos. _Details-on-demand_ refere-se à consulta interativa de mais informações sobre valores selecionados. _Tooltips_ são um meio útil de fornecer detalhes sob demanda. No entanto, as _tooltips_ normalmente só mostram informações para um ponto de dados de cada vez. Como é que podemos mostrar mais?

O _scatter plot_ de classificações de filmes inclui um número de outliers potencialmente interessantes, em que as classificações do Rotten Tomatoes e do IMDB discordam. Vamos criar um gráfico que nos permita selecionar pontos de forma interativa e mostrar as suas etiquetas. Para acionar a consulta de filtro na interação de passar o mouse ou clicar, usaremos o [operador de composição Altair](https://altair-viz.github.io/user_guide/interactions.html#composing-multiple-selections) `|` ("ou").

Passe o mouse sobre os pontos no _scatter plot_ abaixo para ver um destaque e uma etiqueta de título. Clique com a tecla Shift nos pontos para tornar as anotações persistentes e ver várias etiquetas de uma só vez. Que filmes são adorados pelos críticos do Rotten Tomatoes, mas não pelo público em geral no IMDB (ou vice-versa)? Veja se consegue encontrar possíveis erros, onde dois filmes diferentes com o mesmo nome foram acidentalmente combinados!_


In [ ]:
hover = alt.selection_single(
    on='mouseover',  # selecionar ao passar o mouse
    nearest=True,    # selecionar o ponto mais próximo do cursor do mouse
    empty='none'     # a seleção vazia não deve corresponder a nada
)

click = alt.selection_multi(
    empty='none' # partir de seleções vazias sem pontos
)

# codificações de scatterplot compartilhadas por todas as marcas
plot = alt.Chart().mark_circle().encode(
    x='Rotten_Tomatoes_Rating:Q',
    y='IMDB_Rating:Q'
)

# base partilhada para novas camadas
base = plot.transform_filter(
    hover | click # filtrar para pontos em qualquer seleção
)

# pontos do scatter plot de camadas, anotações de halo e etiquetas de título
alt.layer(
    plot.add_selection(hover).add_selection(click),
    base.mark_point(size=100, stroke='firebrick', strokeWidth=1),
    base.mark_text(dx=4, dy=-8, align='right', stroke='white', strokeWidth=2).encode(text='Title:N'),
    base.mark_text(dx=4, dy=-8, align='right').encode(text='Title:N'),
    data=movies
).properties(
    width=600,
    height=450
)

alt.LayerChart(...)

O exemplo acima adiciona três novas camadas ao scatter plot: uma anotação circular, texto branco para proporcionar um fundo legível e texto preto que mostra o título de um filme. Além disso, este exemplo utiliza duas seleções em simultâneo:

1. Uma única seleção (`hover`) que inclui `nearest=True` para selecionar automaticamente o ponto de dados mais próximo à medida que o mouse se move.
2. Uma seleção múltipla (`click`) para criar seleções persistentes via shift-click.

Ambas as seleções incluem o conjunto `empty='none'` para indicar que nenhum ponto deve ser incluído se a seleção estiver vazia. Estas seleções são depois combinadas num único predicado de filtragem &mdash; o _ou_ lógico de `hover` e `click` &mdash; para incluir pontos que residam em _qualquer_ seleção. Utilizamos este predicado para filtrar as novas camadas de modo a mostrar anotações e rótulos apenas para os pontos selecionados.


Utilizando seleções e camadas, podemos realizar uma série de desenhos diferentes para detalhes a pedido! Por exemplo, aqui está uma série temporal em escala logarítmica de preços de ações tecnológicas, anotada com uma linha de orientação e rótulos para a data mais próxima do cursor do mouse:


In [ ]:
# selecionar um ponto para o qual fornecer detalhes a pedido
label = alt.selection_single(
    encodings=['x'], # limitar a seleção ao valor do eixo x
    on='mouseover',  # selecionar em eventos de passagem do mouse
    nearest=True,    # selecionar o ponto de dados mais próximo do cursor
    empty='none'     # a seleção vazia não inclui pontos de dados
)

# definir o nosso gráfico de linha de base dos preços das ações
base = alt.Chart().mark_line().encode(
    alt.X('date:T'),
    alt.Y('price:Q', scale=alt.Scale(type='log')),
    alt.Color('symbol:N')
)

alt.layer(
    base, # gráfico de linhas de base

    # adicionar uma marca de regra para servir de linha de orientação
    alt.Chart().mark_rule(color='#aaa').encode(
        x='date:T'
    ).transform_filter(label),

    # adicionar marcas circulares para pontos temporais selecionados, ocultar pontos não selecionados
    base.mark_circle().encode(
        opacity=alt.condition(label, alt.value(1), alt.value(0))
    ).add_selection(label),

    # adicionar texto com traços brancos para proporcionar um fundo legível para as etiquetas
    base.mark_text(align='left', dx=5, dy=-5, stroke='white', strokeWidth=2).encode(
        text='price:Q'
    ).transform_filter(label),

    # adicionar etiquetas de texto para preços de ações
    base.mark_text(align='left', dx=5, dy=-5).encode(
        text='price:Q'
    ).transform_filter(label),

    data=stocks
).properties(
    width=700,
    height=400
)

alt.LayerChart(...)

_Colocando em ação o que aprendemos até agora: você pode modificar o scatter plot de filmes acima (aquele com a consulta dinâmica ao longo dos anos) para incluir uma marca de regra ("rule") que mostre a classificação média do IMDB (ou Rotten Tomatoes) para os dados contidos na seleção do "intervalo" do ano?



## Brushing &amp; Linking, Revisited

Anteriormente, neste notebook, vimos um exemplo de _brushing &amp; linking_: usando um histograma de consulta dinâmica para destacar pontos em um scatter plot de classificação de filmes. Aqui, visitaremos alguns exemplos adicionais envolvendo seleções vinculadas.

Voltando ao conjunto de dados carros (`cars`), podemos utilizar o operador repetir (`repeat`) para construir uma [matriz de dispersão (SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices) que mostra associações entre quilometragem, aceleração e potência. Podemos definir uma seleção `interval` e incluí-la _dentro_ da nossa especificação de scatter plot repetido para permitir seleções ligadas entre todos os gráficos.

_Clique e arraste em qualquer um dos gráficos abaixo para executar a seleção &amp; ligação!_


In [ ]:
brush = alt.selection_interval(
    resolve='global' # resolver todas as seleções para uma única instância global
)

alt.Chart(cars).mark_circle().add_selection(
    brush
).encode(
    alt.X(alt.repeat('column'), type='quantitative'),
    alt.Y(alt.repeat('row'), type='quantitative'),
    color=alt.condition(brush, 'Cylinders:O', alt.value('grey')),
    opacity=alt.condition(brush, alt.value(0.8), alt.value(0.1))
).properties(
    width=140,
    height=140
).repeat(
    column=['Acceleration', 'Horsepower', 'Miles_per_Gallon'],
    row=['Miles_per_Gallon', 'Horsepower', 'Acceleration']
)

alt.RepeatChart(...)

Note acima a utilização de `resolve='global'` na seleção `interval`. A configuração padrão de `'global'` indica que em todos os gráficos apenas um _brush_ pode estar ativo de cada vez. No entanto, em alguns casos, podemos querer definir _brushes_ em múltiplos gráficos e combinar os resultados. Se utilizarmos `resolve='union'`, a seleção será a _união_ de todos os _brushes_: se um ponto residir em qualquer _brush_ será selecionado. Alternativamente, se utilizarmos `resolve='intersect'`, a seleção consistirá na _intersecção_ de todos os _brushes_: apenas os pontos que residem dentro de todos os _brushes_ serão selecionados.

_Tente definir o parâmetro resolver (`resolve`) como união (`union`) e intersecção (`intersect`) e veja como isso muda a lógica de seleção resultante._


### Filtragem cruzada

Os exemplos de brushes &amp; linking que vimos utilizam toda as codificações condicionais, por exemplo, para alterar os valores de opacidade em resposta a uma seleção. Outra opção é usar uma seleção definida numa vista para _filtrar_ o conteúdo de outra vista.

Vamos construir uma coleção de histogramas para o conjunto de dados voos (`flights`): atraso (`delay`) de chegada (quão cedo ou tarde um vôo chega, em minutos), distância (`distance`) voada (em milhas), e tempo (`time`) de partida (hora do dia). Usaremos o operador `repeat` para criar os histogramas, e adicionaremos uma seleção `interval` para o eixo `x` com brushes resolvidos via intersecção.

Em particular, cada histograma consistirá em duas camadas: uma camada de fundo cinzento e uma camada de primeiro plano azul, com a camada de primeiro plano filtrada pela nossa intersecção de seleções de brushes. O resultado é uma interação de filtragem cruzada entre os três gráficos!

Arraste os intervalos dos brushes nos gráficos abaixo. À medida que seleciona voos com atrasos de chegada mais longos ou mais curtos, como é que as distribuições de distância e tempo respondem?_



In [ ]:
brush = alt.selection_interval(
    encodings=['x'],
    resolve='intersect'
);

hist = alt.Chart().mark_bar().encode(
    alt.X(alt.repeat('row'), type='quantitative',
        bin=alt.Bin(maxbins=100, minstep=1), # até 100 bins
        axis=alt.Axis(format='d', titleAnchor='start') # formato inteiro, título alinhado à esquerda
    ),
    alt.Y('count():Q', title=None) # sem título do eixo y
)

alt.layer(
    hist.add_selection(brush).encode(color=alt.value('lightgrey')),
    hist.transform_filter(brush)
).properties(
    width=900,
    height=100
).repeat(
    row=['delay', 'distance', 'time'],
    data=flights
).transform_calculate(
    delay='datum.delay < 180 ? datum.delay : 180', # atrasos nas pinças > 3 horas
    time='hours(datum.date) + minutes(datum.date) / 60' # horas fracionadas
).configure_view(
    stroke='transparent' # Sem contorno
)

alt.RepeatChart(...)

Ao fazer uma filtragem cruzada, você pode observar que os voos atrasados têm maior probabilidade de partir a horas mais tardias. Este fenómeno é familiar aos passageiros frequentes: um atraso pode propagar-se ao longo do dia, afetando as viagens subsequentes nesse avião. Para ter mais probabilidades de chegar a horas, reserve um voo cedo!_

A combinação de múltiplas visualizações e seleções interativas pode permitir formas valiosas de raciocínio multidimensional, transformando até histogramas básicos em poderosos dispositivos de entrada para colocar questões sobre um dataset!

## Resumo

Para obter mais informações sobre as opções de interação suportadas no Altair, consulte a [documentação de seleção interativa do Altair](https://altair-viz.github.io/user_guide/interactions.html). Para obter detalhes sobre a personalização de manipuladores de eventos, por exemplo, para compor várias técnicas de interação ou suportar a entrada baseada no toque em dispositivos móveis, consulte a [documentação de seleção Vega-Lite](https://vega.github.io/vega-lite/docs/selection.html).

Interessado em saber mais?

- A abstração _seleção_ foi introduzida no artigo [Vega-Lite: A Grammar of Interactive Graphics] (http://idl.cs.washington.edu/papers/vega-lite/), de Satyanarayan, Moritz, Wongsuphasawat, &amp; Heer.
- O sistema PRIM-9 (para projeção, rotação, isolamento e mascaramento em até 9 dimensões) é uma das primeiras ferramentas de visualização interativa, construída no início dos anos 70 por Fisherkeller, Tukey, &amp; Friedman. [Um vídeo de demonstração retrô sobrevive!](https://www.youtube.com/watch?v=B7XoW2qiFUA)
- O conceito de brush & linking foi cristalizado por Becker, Cleveland, &amp; Wilks no seu artigo de 1987 [Dynamic Graphics for Data Analysis](https://scholar.google.com/scholar?cluster=14817303117298653693).
- Para um resumo completo das técnicas de interação para visualização, ver [Interactive Dynamics for Visual Analysis](https://queue.acm.org/detail.cfm?id=2146416) de Heer &amp; Shneiderman.
- Finalmente, para um tratado sobre o que torna a interação eficaz, leia o artigo clássico [Diret Manipulation Interfaces](https://scholar.google.com/scholar?cluster=15702972136892195211) de Hutchins, Hollan, &amp; Norman.

#### Apêndice: Sobre a representação do tempo

Anteriormente, observamos um pequeno aumento no número de filmes em 1969 e 1970. De onde é que vem esse aumento? E porquê 1969 _ou_ 1970? A resposta resulta de uma combinação de dados em falta e da forma como o seu computador representa o tempo.

Internamente, as datas e horas são representadas relativamente à [época UNIX](https://en.wikipedia.org/wiki/Unix_time), em que o tempo "zero" corresponde ao toque da meia-noite de 1 de janeiro de 1970 na [hora UTC](https://en.wikipedia.org/wiki/Coordinated_Universal_Time), que corre ao longo do [meridiano principal](https://en.wikipedia.org/wiki/Prime_meridian). Acontece que há alguns filmes com datas de lançamento em falta (`null`). Esses valores nulos (`null`) são interpretados como hora `0` e, portanto, mapeados para 1 de janeiro de 1970 na hora UTC. Se vive nas Américas &ndash; e, portanto, em fusos horários "anteriores" &ndash; este ponto preciso no tempo corresponde a uma hora anterior a 31 de dezembro de 1969 no seu fuso horário local. Por outro lado, se vive perto ou a leste do meridiano principal, a data no seu fuso horário local será 1 de janeiro de 1970.

A conclusão? Seja sempre cético em relação aos seus dados e tenha em atenção que a forma como os dados são representados (seja como datas e horas, floating point numbers, latitudes e longitudes, etc.) pode, por vezes, levar a artefatos que afetam a análise!


